In [54]:
#importing dependencies and setting app token
import pandas as pd
import requests
from datetime import date, datetime
from sodapy import Socrata
import csv
from io import StringIO
from sqlalchemy import create_engine

MyAppToken = 'GuILMuJLLhVOQ8u9cyXPc56p3'

#checking the day of the month and printing the result, this is used to filter the dataframe later
today = date.today()
day = today.strftime("%d")
print("Day =", day)

Day = 14


In [31]:
#still need a way to filter 
client = Socrata("data.cityofchicago.org", MyAppToken, None, None )

#getting () results from the soda api
try:
    results = client.get("ijzp-q8t2", limit=20000)
                         
except requests.exceptions.Timeout:
  print("Timeout occurred")

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [55]:
#dropping columns that have no use
clean_columns_df = results_df.drop(columns=['year','location','x_coordinate', 'y_coordinate','id','beat','district','ward','community_area','fbi_code','updated_on', 'case_number', 'block', 'iucr', 'location_description', 'arrest', ':@computed_region_awaf_s7ux', ':@computed_region_6mkv_f3dw', ':@computed_region_vrxf_vc4k', ':@computed_region_bdys_3d7i', ':@computed_region_43wa_7qmu' , ':@computed_region_rpca_8um6',':@computed_region_d9mm_jgwp',':@computed_region_d3ds_rm58'])
#formatting datetime to ease readability
clean_columns_df['date'] = pd.to_datetime(clean_columns_df['date']).dt.strftime('%m.%d.%Y')
clean_columns_df['month'] = pd.to_datetime(clean_columns_df['date']).dt.strftime('%m')
clean_columns_df.head()

,date,primary_type,description,domestic,latitude,longitude,month
0,04.06.2020,THEFT,$500 AND UNDER,False,42.018734342,-87.66588196,04
1,04.06.2020,BATTERY,DOMESTIC BATTERY SIMPLE,True,41.790019182,-87.65841048,04
2,04.06.2020,BATTERY,AGGRAVATED - HANDGUN,False,41.791021009,-87.625158623,04
3,04.06.2020,THEFT,OVER $500,False,41.969349668,-87.725634004,04
4,04.06.2020,MOTOR VEHICLE THEFT,AUTOMOBILE,False,NaN,NaN,04


In [58]:
#reorganizing columns to read easier left to right
clean_columns_df = clean_columns_df[["date", 'month','primary_type','description','domestic',"latitude","longitude"]]
domestic_crimes = clean_columns_df['domestic'] == True
non_domestic_crimes = clean_columns_df['domestic'] == False
domestic_crimes_df = clean_columns_df[domestic_crimes]
non_domestic_crimes_df = clean_columns_df[non_domestic_crimes]

In [59]:
#still have to use day to filter results from last year
non_domestic_crimes_df

,date,month,primary_type,description,domestic,latitude,longitude
0,04.06.2020,04,THEFT,$500 AND UNDER,False,42.018734342,-87.66588196
2,04.06.2020,04,BATTERY,AGGRAVATED - HANDGUN,False,41.791021009,-87.625158623
3,04.06.2020,04,THEFT,OVER $500,False,41.969349668,-87.725634004
4,04.06.2020,04,MOTOR VEHICLE THEFT,AUTOMOBILE,False,NaN,NaN
5,04.06.2020,04,BATTERY,AGGRAVATED - HANDGUN,False,42.014981521,-87.667036193
...,...,...,...,...,...,...,...
19994,02.27.2020,02,ROBBERY,ARMED - HANDGUN,False,41.890427259,-87.76339231
19995,02.27.2020,02,BATTERY,SIMPLE,False,41.862218265,-87.639087164
19997,02.27.2020,02,CRIMINAL DAMAGE,TO VEHICLE,False,41.822133142,-87.609788613
19998,02.27.2020,02,MOTOR VEHICLE THEFT,AUTOMOBILE,False,41.908873227,-87.721630446


In [51]:
#making dictionaries for mongoDB
non_domestic_chicago_data = non_domestic_crimes_df.to_dict()
domestic_chicago_data = domestic_crimes_df.to_dict()

In [60]:
def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

rds_connection_string = "postgres:postgres@localhost:5432/crimeandcovid19"
engine = create_engine(f'postgresql://{rds_connection_string}')
clean_columns_df.to_sql('chicago_data', engine, method=psql_insert_copy)

OperationalError: (psycopg2.OperationalError) could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused (0x0000274D/10061)
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?

(Background on this error at: http://sqlalche.me/e/e3q8)

In [ ]:
engine.table_names()

In [ ]:
pd.read_sql_query('select * from chicago_data', con=engine).head()